In [2]:
!git config --global user.name "AlexRomero01"
!git config --global user.email "alexromerosegues@gmail.com"
!git clone https://github.com/AlexRomero01/TRAFFIC-SIGN-DETECTION-AND-IDENTIFICATION.git


Cloning into 'TRAFFIC-SIGN-DETECTION-AND-IDENTIFICATION'...


# Libraries

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
import tensorflow as tf
from PIL import Image
import os
import keras
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
from keras.models import Sequential, load_model
from keras.layers import Conv2D, MaxPool2D, Dense, Flatten, Dropout
from google.colab import drive
from matplotlib import style
import random
style.use('fivethirtyeight')



from tensorflow import keras
from PIL import Image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import accuracy_score
np.random.seed(42)

# Data storage

In [ ]:
# Montar Google Drive
drive.mount('/content/drive')
dataset_path = '/content/drive/MyDrive/TFG CODE/DATASET SIGNALS'
os.chdir = dataset_path

train_path = '/content/drive/MyDrive/TFG CODE/DATASET SIGNALS/Train'
test_path = '/content/drive/MyDrive/TFG CODE/DATASET SIGNALS/Test'
data_dir = '/content/drive/MyDrive/TFG CODE/DATASET SIGNALS'

IMG_HEIGHT = 30
IMG_WIDTH = 30
channels = 3

NUM_CATEGORIES = len(os.listdir(train_path))
data = []
labels = []
# We have 43 different kinds of signals
classes = 43
cur_path = os.getcwd()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Useful paths for later on

In [ ]:
class_names = { 0:'Speed limit (20km/h)',
            1:'Speed limit (30km/h)',
            2:'Speed limit (50km/h)',
            3:'Speed limit (60km/h)',
            4:'Speed limit (70km/h)',
            5:'Speed limit (80km/h)',
            6:'End of speed limit (80km/h)',
            7:'Speed limit (100km/h)',
            8:'Speed limit (120km/h)',
            9:'No passing',
            10:'No passing veh over 3.5 tons',
            11:'Right-of-way at intersection',
            12:'Priority road',
            13:'Yield',
            14:'Stop',
            15:'No vehicles',
            16:'Veh > 3.5 tons prohibited',
            17:'No entry',
            18:'General caution',
            19:'Dangerous curve left',
            20:'Dangerous curve right',
            21:'Double curve',
            22:'Bumpy road',
            23:'Slippery road',
            24:'Road narrows on the right',
            25:'Road work',
            26:'Traffic signals',
            27:'Pedestrians',
            28:'Children crossing',
            29:'Bicycles crossing',
            30:'Beware of ice/snow',
            31:'Wild animals crossing',
            32:'End speed + passing limits',
            33:'Turn right ahead',
            34:'Turn left ahead',
            35:'Ahead only',
            36:'Go straight or right',
            37:'Go straight or left',
            38:'Keep right',
            39:'Keep left',
            40:'Roundabout mandatory',
            41:'End of no passing',
            42:'End no passing veh > 3.5 tons' }

Very important part. A loop that checks all the data inside the train folder and labels it correctly and stores everything in each list (data and labels) created before. 1 by 1 it resizes the image to 30x30 pixels so later we will have no problems when training the AI with these images.

# Data Processing


In [ ]:
for i in range(classes):
    path = os.path.join(dataset_path, 'Train', str(i))
    images = os.listdir(path)
    for a in images:
        try:
            image = Image.open(os.path.join(path, a))
            image = image.resize((IMG_HEIGHT, IMG_WIDTH))
            image = np.array(image)
            data.append(image)
            labels.append(i)
        except Exception as e:
            print(e)

In [ ]:
# Convertir listas a numpy arrays
data = np.array(data)
labels = np.array(labels)

In [ ]:
print(data.shape, labels.shape)

(39241, 30, 30, 3) (39241,)


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=0)

In [ ]:
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(31392, 30, 30, 3) (7849, 30, 30, 3) (31392,) (7849,)


In [ ]:
y_train = to_categorical(y_train, 43)
y_test = to_categorical(y_test, 43)

Splitting the data into train and validation set

In [ ]:
shuffle_indexes = np.arange(data.shape[0])
np.random.shuffle(shuffle_indexes)
data = data[shuffle_indexes]
labels = labels[shuffle_indexes]

In [ ]:
random_state=random.randint(1,10000)
X_train, X_val, y_train, y_val = train_test_split(data, labels, test_size=0.3, random_state=42, shuffle=True)   #random.randint(1,10000)

X_train = X_train/255
X_val = X_val/255

print("Random: ", random_state)
print("X_train.shape", X_train.shape)
print("X_valid.shape", X_val.shape)
print("y_train.shape", y_train.shape)
print("y_valid.shape", y_val.shape)

Random:  5385
X_train.shape (27468, 30, 30, 3)
X_valid.shape (11773, 30, 30, 3)
y_train.shape (27468,)
y_valid.shape (11773,)


One hot encoding the labels

In [ ]:
y_train = keras.utils.to_categorical(y_train, classes)
y_val = keras.utils.to_categorical(y_val, classes)

print(y_train.shape)
print(y_val.shape)

(27468, 43)
(11773, 43)


# Model


In [ ]:
!pip install keras-tuner


In [ ]:
aug = ImageDataGenerator(
    rotation_range=10,
    zoom_range=0.15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.15,
    horizontal_flip=False,
    vertical_flip=False,
    fill_mode="nearest")



In [ ]:
import kerastuner as kt

def model_builder(hp):

    model = keras.Sequential()

    hp_filters1 = hp.Int('filters1', min_value=16, max_value=128, step=16)
    model.add(keras.layers.Conv2D(filters=hp_filters1, kernel_size=(3, 3), activation='relu', input_shape=(IMG_HEIGHT, IMG_WIDTH, channels)))

    hp_filters2 = hp.Int('filters2', min_value=32, max_value=256, step=32)
    model.add(keras.layers.Conv2D(filters=hp_filters2, kernel_size=(3, 3), activation='relu'))

    hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])

    model.add(keras.layers.MaxPooling2D(pool_size=(2, 2)))
    model.add(keras.layers.BatchNormalization(axis=-1))

    model.add(keras.layers.Conv2D(filters=64, kernel_size=(3,3), activation='relu'))
    model.add(keras.layers.Conv2D(filters=128, kernel_size=(3,3), activation='relu'))
    model.add(keras.layers.MaxPooling2D(pool_size=(2, 2)))
    model.add(keras.layers.BatchNormalization(axis=-1))
    model.add(keras.layers.Flatten())

    hp_units = hp.Int('units', min_value=256, max_value=1024, step=256)
    model.add(keras.layers.Dense(units=hp_units, activation='relu'))
    model.add(keras.layers.BatchNormalization())
    model.add(keras.layers.Dropout(rate=0.5))
    model.add(keras.layers.Dense(43, activation='softmax'))

    model.compile(optimizer=keras.optimizers.Adam(learning_rate=hp_learning_rate),
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])

    return model


tuner = kt.Hyperband(model_builder,
                     objective='val_accuracy',
                     max_epochs=10,

                     factor=3,
                     directory='my_dir',
                     project_name='traffic_sign_recognition')

stop_early = keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

tuner.search(aug.flow(X_train, y_train, batch_size=32),
             epochs=30,
             validation_data=(X_val, y_val),
             callbacks=[stop_early])

best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

print(f"The optimal number of filters in the first Conv2D layer is {best_hps.get('filters1')}")
print(f"The optimal number of filters in the second Conv2D layer is {best_hps.get('filters2')}")
print(f"The optimal learning rate for the optimizer is {best_hps.get('learning_rate')}")
print(f"The optimal number of units in the Dense layer is {best_hps.get('units')}")


In [ ]:
model.summary()

In [ ]:
model = tuner.hypermodel.build(best_hps)
history = model.fit(aug.flow(X_train, y_train, batch_size=32), epochs=30
                    , validation_data=(X_val, y_val))

In [ ]:
save_path = '/content/drive/MyDrive/TFG CODE/model.h5'
model.save(save_path)

# Results


In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(epochs)

plt.figure(figsize=(10, 10))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

In [ ]:
test = pd.read_csv(dataset_path + '/Test.csv')

labels = test["ClassId"].values
imgs = test["Path"].values

data =[]

for img in imgs:
    try:
        image = cv2.imread(data_dir + '/' +img)
        image_fromarray = Image.fromarray(image, 'RGB')
        resize_image = image_fromarray.resize((IMG_HEIGHT, IMG_WIDTH))
        data.append(np.array(resize_image))
    except:
        print("Error in " + img)
X_test = np.array(data)
X_test = X_test/255
pred_probs = model.predict(X_test)
pred_classes = np.argmax(pred_probs, axis=1)

#Accuracy with the test data
print('Test Data accuracy: ',accuracy_score(labels, pred_classes)*100)

In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns
from sklearn.metrics import classification_report

cf = confusion_matrix(labels, pred)
df_cm = pd.DataFrame(cf, index = classes,  columns = classes)
plt.figure(figsize = (20,20))
# annot = True will display value associated with each cell
sns.heatmap(df_cm, annot=True)

print(classification_report(labels, pred))

# New Images Tets


In [ ]:
Raw_images_path = '/content/drive/MyDrive/TFG CODE/Raw_Images/'
new_image = cv2.imread(Raw_images_path + '4.webp')
print(new_image.shape)

In [ ]:
plt.imshow(cv2.cvtColor(new_image, cv2.COLOR_BGR2RGB))
plt.show()

In [ ]:
gray_image = cv2.cvtColor(new_image, cv2.COLOR_BGR2GRAY)
plt.imshow(gray_image)
plt.show()
print("Shape: ",gray_image.shape)

In [ ]:
blurred_image = cv2.GaussianBlur(gray_image, (5, 5), 0)
plt.imshow(blurred_image)
plt.show()

In [ ]:
batch_size_new=1
channels_new=3
height_new=30
width_new=30
resized_image = cv2.resize(new_image, (width_new, height_new))
final_image = np.reshape(resized_image, (batch_size_new, height_new, width_new, channels_new))

In [ ]:
print("Shape: ",final_image.shape)

In [ ]:
predictions = model.predict(final_image)
print(predictions)

In [ ]:
class_name, conf_score = classify(image, model, class_names)